# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Getting data. 

## Now we are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Task 1. Getting the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
###### FOR THIS QUERY ITEM_IN_SESSION AND SESSION_ID ARE CONSIDERED AS THE COMPOSITE KEY, BECAUSE THEIR COMBINATION FILTERS THE DATA AS PER THE QUERY WE ARE ANSWERING FROM THE TABLE.

In [8]:
# Create table
query1 = "CREATE TABLE IF NOT EXISTS song_playlist_session_item"
query1 = query1 + "(session_id int, item_in_session int, artist text, song_title text, song_length float, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query1)
except Exception as e:
    print(e)                    

In [9]:
# set up the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# INSERT data in the table
        query = "INSERT INTO song_playlist_session_item (session_id, item_in_session, artist, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]),int(line[3]),line[0],line[9],float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [14]:
query = "select artist, song_title, song_length from song_playlist_session_item WHERE session_id = 338 and item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Task 2. Getting the name of artist, the song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
###### IN THIS QUERY WE USED SESSION_ID AND USER_ID AS THE PARTITION KEY FOR GETTING NEEDED DATA AND THE ITEM_IN_SESSION AS THE CLUSTERING COLUMN TO FULFILL THE SORTING REQUIREMENT.

In [15]:
# Create table
query2 = "CREATE TABLE IF NOT EXISTS song_playlist_user_session"
query2 = query2 + "(session_id int, user_id int, item_in_session int, artist text, song_title text, user_FirstName text, user_LastName text, \
                    PRIMARY KEY ((session_id, user_id), item_in_session))"
try:
    session.execute(query2)
except Exception as e:
    print(e)
                    

In [17]:
# set up the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# INSERT data in the table
        query = "INSERT INTO song_playlist_user_session (session_id, user_id, item_in_session, artist, song_title, user_FirstName, user_LastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]),int(line[10]),int(line[3]),line[0],line[9],line[1],line[4]))
                    

#### Do a SELECT to verify that the data have been inserted into each table

In [18]:
query = "select artist, song_title, user_FirstName, user_LastName from song_playlist_user_session WHERE session_id = 182 and user_id = 10 order by item_in_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.user_firstname, row.user_lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Task 3. Getting the user name (first and last) in music app history who listened to the song 'All Hands Against His Own'
###### IN THIS QUERY WE USED SONG AS THE PARTITION KEY FOR GETTING NEEDED DATA AND THE USERID AS THE CLUSTERING COLUMN TO UNIQUELY IDENTIFY EACH RECORD PER SONG PER USER.

In [20]:
# Create table
Query3 = "CREATE TABLE IF NOT EXISTS song_playlist_songTitle"
Query3 = Query3 + "(song text, userId int,firstName text, lastName text, PRIMARY KEY ((song),userId))"
 
try:
    session.execute(Query3)
except Exception as e:
    print(e)


In [21]:
# set up the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# INSERT data in the table
        query = "INSERT INTO song_playlist_songTitle (song,userId,firstName,lastName)"
        query = query + " VALUES (%s,%s,%s,%s)"
        session.execute(query, (line[9],int(line[10]),line[1],line[4]))

#### Do a SELECT to verify that the data have been inserted into each table

In [22]:
query = "SELECT firstName, lastName from song_playlist_songTitle WHERE song = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row.firstname,row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [23]:
drop1 = "DROP TABLE song_playlist_session_item"
try:
    rows = session.execute(drop1)
except Exception as e:
    print(e)
    
drop2 = "DROP TABLE song_playlist_user_session"
try:
    rows = session.execute(drop2)
except Exception as e:
    print(e)

drop3 = "DROP TABLE song_playlist_songTitle"
try:
    rows = session.execute(drop3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [24]:
session.shutdown()
cluster.shutdown()